In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, signal
from scipy.fft import fft, fftfreq
import os
from collections import defaultdict
import seaborn as sns
from tqdm import tqdm, trange
#from google.colab import drive
import glob
import pyarrow.parquet as pq
#drive.mount('/content/drive')
%matplotlib inline

plt.rcParams.update(plt.rcParamsDefault)

plt.rcParams['pdf.fonttype']= 42
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 11
plt.rcParams["text.color"] = 'black'

In [ ]:
p = 'H:/.shortcut-targets-by-id/10pxdlRXtzFB-abwDGi0jOGOFFNm3pmFK/Tuthill Lab Shared/Jessica/'
data1 = pd.read_csv(p + 'all_data_headvsheadless_pub.pq')
data2 = pd.read_csv(p + 'all_head_on_fictracdata.pq')

In [ ]:
def plot_heading_histogram(angles_deg, bins=36, bottom=0, alpha=0.5, color='orchid', edgecolor='k', proportion=False, hide_y=False, set_bottom=False):
    angles_rad = np.deg2rad(angles_deg % 360)
    counts, bin_edges = np.histogram(angles_rad, bins=bins, range=(0, 2*np.pi))
    bin_width = bin_edges[1] - bin_edges[0]
    
    if proportion:
        counts = counts / counts.sum()

    if set_bottom:
        bottom = np.max(counts)

    fig = plt.figure(figsize=[3, 3])
    ax = fig.add_subplot(1, 1, 1, projection="polar")
    ax.bar(bin_edges[:-1], counts, width=bin_width, bottom=bottom, alpha=alpha, color=color, edgecolor=edgecolor, linewidth=.3)
    
    if hide_y:
        ax.set_yticklabels([])
        ax.yaxis.grid(False)


    return fig, ax

f, ax = plot_heading_histogram(data2[(data2.dir!='control')&
                                           (data2.stimlen!=0.0)&
                                           (data2.fictrac_delta_rot_lab_y_mms.between(1,30))&
                                           (data2.fnum.between(0.499, 1.5))]
                                            ['fictrac_heading_deg'].values, 
                                           color="#000000",
                                           bins=60, bottom=0, proportion=True, hide_y=True, set_bottom=True)
plt.show()




In [ ]:
from scipy.stats import ttest_ind

def compare_heading_vectors(group1_angles_deg, group2_angles_deg, bins=36):
    """
    Compare the mean resultant vector directions of two groups of angles (in degrees).
    Returns the mean direction and resultant length for each group.
    """
    def mean_vector(angles_deg):
        angles_rad = np.deg2rad(angles_deg % 360)
        # Compute mean resultant vector
        x = np.cos(angles_rad).mean()
        y = np.sin(angles_rad).mean()
        mean_angle = np.arctan2(y, x)
        mean_angle_deg = np.rad2deg(mean_angle) % 360
        r = np.sqrt(x**2 + y**2)
        return mean_angle_deg, r

    mean1, r1 = mean_vector(group1_angles_deg)
    mean2, r2 = mean_vector(group2_angles_deg)

    print(f"Group 1: Mean direction = {mean1:.2f}°, Resultant length = {r1:.2f}")
    print(f"Group 2: Mean direction = {mean2:.2f}°, Resultant length = {r2:.2f}")

    return (mean1, r1), (mean2, r2)

# Example usage:
(mean1, r1), (mean2, r2) = compare_heading_vectors(
    data2[(data2.dir=='control')&
                (data2.stimlen!=0.0)&
                (data2.fictrac_delta_rot_lab_y_mms.between(1,30))&
                (data2.fnum.between(0.499, 1.5))]
                ['fictrac_heading_deg'].values,
    data2[(data2.dir!='control')&
                (data2.stimlen!=0.0)&
                (data2.fictrac_delta_rot_lab_y_mms.between(1,30))&
                (data2.fnum.between(0.499, 1.5))]
                ['fictrac_heading_deg'].values
)

# Compute the difference in means
mean_diff = abs(mean1 - mean2)
print(f"Difference in resultant means: {mean_diff:.4f}°")

# Get the angle distributions (in degrees)
group1 = data2[(data2.dir=='control')&
                     (data2.stimlen!=0.0)&
                     (data2.fictrac_delta_rot_lab_y_mms.between(1,30))&
                     (data2.fnum.between(0.499, 1.5))]['fictrac_heading_deg'].values

group2 = data2[(data2.dir!='control')&
                     (data2.stimlen!=0.0)&
                     (data2.fictrac_delta_rot_lab_y_mms.between(1,30))&
                     (data2.fnum.between(0.499, 1.5))]['fictrac_heading_deg'].values

# A better test for comparing circular (angle) data is the Watson-Williams test or the nonparametric Watson's U^2 test.
# You can use scipy.stats.circmean/circstd for summary, but for statistical comparison, use pycircstat if available.
# Example with pycircstat (if installed):

# import pycircstat
# import pycircstat.tests
# p_val = pycircstat.tests.watson_williams(np.deg2rad(group1), np.deg2rad(group2))
# print(f"Watson-Williams test p = {p_val:.4f}")

# If pycircstat is not available, you can use scipy.stats.circmean for summary statistics:
circmean1 = stats.circmean(np.deg2rad(group1), high=2*np.pi)
circmean2 = stats.circmean(np.deg2rad(group2), high=2*np.pi)
print(f"Group 1 circular mean: {np.rad2deg(circmean1):.2f}°")
print(f"Group 2 circular mean: {np.rad2deg(circmean2):.2f}°")
# compare two means with t-test (not ideal for circular data)
t_stat, p_val = ttest_ind(group1, group2)
print(f"T-test p = {p_val:.4f}")
print("significant?:",p_val<=0.01)

In [ ]:
fig = plt.figure(figsize=[1.5,2.0])
sns.lineplot(data=data2[(data2.stimlen.isin([1.0]))&(data2.dir.isin(['middle', 'control']))&(data.fictrac_delta_rot_lab_y_mms<=30)], 
                   hue='dir', x='fnum', y='fictrac_delta_rot_lab_y_mms', errorbar=('ci', 95),
                  legend=False,
                  palette=[ "#3CC443",'#000000'])


plt.yticks([0,5,10])
plt.xticks([0,0.5,1,1.5,2])
plt.ylabel('Velocity (mm/s)')
plt.xlabel('Time (s)')
sns.despine(trim=True)
plt.show()

In [ ]:
# Function to classify behavior based on velocity in head-on
def classify_behavior(velocity):
    if velocity < 1:
        return 'standing'

    elif velocity >= 1 and velocity < 2:
        return 'grooming/other'
    
    elif velocity >= 2 and velocity <= 35:
        return 'walking'

    elif velocity > 35:
        return 'jumping'
    else:
        return 'unknown'  # Handle any unexpected values
# Apply the function to create the 'behavior' column
splits = ['5_0', '7_0', '8_0', '9_0']
mid = data1[data1.date_parsed=='4.28.22']
#mid = mid[~mid.fly.isin(splits)]
d = mid.copy()
d['behavior'] = d['fictrac_delta_rot_lab_y_mms'].apply(classify_behavior)

behavior_counts_head = d.groupby(['fnum','stimlen','behavior', 'fly']).size().reset_index(name='count')
total_counts_head = d.groupby(['fnum','stimlen', 'fly']).size().reset_index(name='total_count')
# Merge the behavior counts with total counts
merged_head = pd.merge(behavior_counts_head, total_counts_head, on=['fnum','stimlen', 'fly'])
# Calculate the probability of each behavior at each frame
merged_head['probability'] = merged_head['count'] / merged_head['total_count']
merged_head['probability'] = merged_head['probability'] *100

In [ ]:
fig = plt.figure(figsize=(2.1, 1.5))
sns.lineplot(data=d[d.behavior.isin(['jumping'])][d.stimlen!=0.0], 
           x='fnum', 
           y='jumping_prob', 
           hue='behavior', 
           palette=["#000000"],
           #errorbar=None,
           legend=False,)
plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks(np.arange(0.0, 0.26, .05))

sns.despine(trim=True)
plt.show()

In [ ]:
# Define stimulus onset (if known/fixed)
stimulus_onset_frame = 0.499  # Example value

# Filter to only relevant data after stimulus onset
df_after_stimulus = d[d["fnum"] >= stimulus_onset_frame]

# Group by fullfile and fly
latency_results = []

for (fullfile, fly), group in df_after_stimulus.groupby(["fullfile", "fly"]):
    jumping_rows = group[group["behavior"] == "jumping"]
    
    if not jumping_rows.empty:
        first_jump_frame = jumping_rows["fnum"].min()
        latency = first_jump_frame - stimulus_onset_frame
    else:
        latency = None  # Or some sentinel value like -1 or np.nan

    latency_results.append({
        "fullfile": fullfile,
        "fly": fly,
        "latency_to_jump": latency
    })

# Convert results to DataFrame
latency_df = pd.DataFrame(latency_results)

latency_df.dropna()
fig=plt.figure(figsize=[1,2])
sns.stripplot(data=latency_df.dropna(),  
              y='latency_to_jump',
              size=3,
              alpha=.5,
              color='black')
sns.despine(trim=True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(2.1, 1.5))
sns.lineplot(data=d[d.behavior.isin(['walking'])][d.stimlen!=0.0], 
           x='fnum', 
           y='walking_prob', 
           hue='behavior', 
           palette=["#2868c7"],
           #errorbar=None,
           legend=False,)
plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks([0.5,0.55,.6,.65,])
plt.ylim(0.5, .65)
sns.despine(trim=True)
plt.show()

In [ ]:
# Define stimulus onset (if known/fixed)
stimulus_onset_frame = 0.499  # Example value

# Filter to only relevant data after stimulus onset
df_after_stimulus = d[d["fnum"] >= stimulus_onset_frame]

# Group by fullfile and fly
latency_results = []

for (fullfile, fly), group in df_after_stimulus.groupby(["fullfile", "fly"]):
    walking_rows = group[group["behavior"] == "walking"]
    
    if not walking_rows.empty:
        first_walking_frame = walking_rows["fnum"].min()
        latency = first_walking_frame - stimulus_onset_frame
    else:
        latency = None  # Or some sentinel value like -1 or np.nan

    latency_results.append({
        "fullfile": fullfile,
        "fly": fly,
        "latency_to_walk": latency
    })

# Convert results to DataFrame
latency_df = pd.DataFrame(latency_results)

latency_df.dropna()
fig=plt.figure(figsize=[1,2])
sns.stripplot(data=latency_df.dropna(),  
              y='latency_to_walk',
              size=3,
              alpha=.5,
              color="#2868c7",
                )
sns.despine(trim=True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(2.1, 1.5))
sns.lineplot(data=d[d.behavior.isin(['grooming/other'])][d.stimlen!=0.0], 
           x='fnum', 
           y='t3_grooming_prob', 
           hue='behavior', 
           palette=["#fc8be6"],
           #errorbar=None,
           legend=False,)
plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks([0.0, 0.1, 0.2])

sns.despine(trim=True)
plt.show()

fig = plt.figure(figsize=(2.1, 1.5))
sns.lineplot(data=d[d.behavior.isin(['standing'])][d.stimlen!=0.0], 
           x='fnum', 
           y='standing_prob', 
           hue='behavior', 
           palette=["#B6B4B4"],
           #errorbar=None,
           legend=False,)
plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks([0.05, .1,.15,.2])
plt.ylim(0.05, .25)
sns.despine(trim=True)
plt.show()

In [ ]:

behavior_map = {behavior: idx for idx, behavior in enumerate(d['behavior'].unique())}
d['behavior_num'] = d['behavior'].map(behavior_map)


# Group by 'genotype' and 'time', then count occurrences of each behavior
grouped = d.groupby(['filename','fly', 'fnum', 'behavior']).size().reset_index(name='count')

# Calculate total counts for each genotype and time combination
total_counts = grouped.groupby(['filename','fly', 'fnum'])['count'].sum().reset_index(name='total_count')

# Merge total counts back to grouped dataframe to calculate probabilities
grouped = pd.merge(grouped, total_counts, on=['filename','fly', 'fnum'])

# Calculate probability as count divided by total_count
grouped['probability'] = grouped['count'] / grouped['total_count']

# Select relevant columns for the final dataframe
probabilities_df = grouped[['filename','fly', 'fnum', 'behavior', 'probability']]


# Sample DataFrame (Replace this with your actual DataFrame)
# df = pd.read_csv('your_data.csv')

# Step 1: Convert 'behavior' to numerical representation
behavior_map = {behavior: idx for idx, behavior in enumerate(probabilities_df['behavior'].unique())}
probabilities_df['behavior_num'] = probabilities_df['behavior'].map(behavior_map)
probabilities_df = probabilities_df[probabilities_df.behavior!='unknown']
# Step 2: Plot Ethograms for Each Unique file_path and fly_id
unique_file_paths = probabilities_df['filename'].unique()

from matplotlib.colors import ListedColormap

#{'walking': 0, 'grooming/other': 1, 'standing': 2, 'jumping': 3}

custom_cmap = ListedColormap(["#2868c7", "#e268f2", "#ffffff", "#000000", "#ffffff"])


fly_df = probabilities_df[probabilities_df.filename.str.contains('-1 sec')]
unique_file_ids = np.random.permutation(fly_df['filename'].unique())
fly_df['filename'] = pd.Categorical(fly_df['filename'], categories=unique_file_ids, ordered=True)

pivot_df = fly_df.pivot_table(index='filename', columns='fnum', values='behavior_num', aggfunc=lambda x: x.mode()[0])

fig = plt.figure(figsize=(5.5, len(unique_file_ids) * 0.05))
sns.heatmap(pivot_df, cmap=custom_cmap, vmax=4, cbar_kws={'label': 'Behavior'}, xticklabels=150, yticklabels=1)
plt.xlabel('Time (s)')
plt.ylabel('Fly ID')
plt.show()

In [ ]:
headless_data = data1[data1['head'].isin(['OFF', 'other'])]
headless_data = headless_data[~headless_data.fly.isin(['10_0','11_0', '12_0', '13_0', '14_0', '15_0', '6_0', '7_0', '8_0', '9_0'])]
headless_data = headless_data[headless_data.date_parsed.isin(['3.24.23', '3.28.23', '5.11.22'])]


from scipy.signal import find_peaks
def detect_peaks(series, min_val, max_val, time_series=None):
    """
    Detect peaks in a time series that exceed a threshold above a baseline 
    calculated from the first 0.25 seconds of the data.
    
    Parameters:
        series (pd.Series): The data series (e.g., joint angle or kinematic data).
        min_val (float): Minimum peak height (optional).
        max_val (float): Maximum peak height (optional).
        time_series (pd.Series or np.ndarray): Optional time values aligned with `series`.
    
    Returns:
        np.ndarray: Indices of peaks that exceed 25 units above baseline.
    """
    # def detect_peaks(series, min_val, max_val):
    # """
    # Detect if a time series oscillates between given min and max values.
    # """
    # # Find peaks and troughs
    # if min_val and max_val  is None:
    #      peaks, _ = find_peaks(series)
    # else:
    #     peaks, _ = find_peaks(series, height=(min_val, max_val))
    # #troughs, _ = find_peaks(-series, distance=30)
    
    # # Check if peaks and troughs fall within the desired range
    # return peaks
    # Find all candidate peaks
    if min_val is None and max_val is None:
        peaks, properties = find_peaks(series)
    else:
        peaks, properties = find_peaks(series, height=(min_val, max_val))
    
    # Determine baseline from first 0.25 seconds
    if time_series is not None:
        baseline_mask = time_series <= 0.25
        baseline = series[baseline_mask].mean()
    else:
        # Assume uniform sampling and take first 25% of 1 second (adjust if needed)
        baseline_window = int(len(series) * 0.25)  # Modify if sampling rate known
        baseline = series.iloc[:baseline_window].mean()

    # Keep only peaks where height > baseline + 25
    peak_heights = series[peaks]
    valid_peak_indices = peaks[peak_heights > (baseline + 25)]
    
    return valid_peak_indices

def classify_behavior(peaks_detected):
    """
    Classify behavior based on detected peaks in leg kinematics.
    """
    kinematic_cols = ['L1_FTi', 'L2_FTi', 'L3_FTi', 'R1_FTi', 'R2_FTi', 'R3_FTi']
    
    # Check which columns have peaks
    peaks_present = {col: len(peaks_detected[col]) > 0 for col in kinematic_cols}
    
    # Determine behavior based on detected peaks
    # if all(peaks_present[col] for col in ['L1_FTi', 'R1_FTi', 'L3_FTi', 'R3_FTi']):
    #     return 'walking'
    if peaks_present['L3_FTi'] and peaks_present['R3_FTi'] and not any(peaks_present[col] for col in ['L1_FTi', 'R1_FTi', 'L2_FTi', 'R2_FTi']):
        return 'grooming'
    elif peaks_present['L2_FTi'] and peaks_present['R2_FTi'] and not any(peaks_present[col] for col in ['L1_FTi', 'R1_FTi', 'L3_FTi', 'R3_FTi']):
        return 'grooming'
    elif sum(peaks_present[col] for col in kinematic_cols) == 1:
        return 'kicking'
    else:
        return 'standing'

def process_chunks(trial_df, min_peak_height, max_peak_height):
    """
    Process each chunk to classify behavior based on peak detection.
    """
    chunk_behavior = []
    
    # Identify continuous chunks of time where peaks are detected
    kinematic_cols = ['L1_FTi', 'L2_FTi', 'L3_FTi', 'R1_FTi', 'R2_FTi', 'R3_FTi']
    
    for start in range(0, len(trial_df), 35):  # Adjust chunk size as needed
        end = min(start + 35, len(trial_df))  # Define chunk end
        chunk_df = trial_df.iloc[start:end]
        
        # Detect peaks in each column within the chunk
        # peaks_detected = {col: detect_peaks(chunk_df[col], min_peak_height, max_peak_height) for col in kinematic_cols}
        # Ensure chunk_df index is reset so peak indices align with chunk_df rows
        chunk_df = chunk_df.reset_index(drop=True)
        peaks_detected = {
            col: detect_peaks(chunk_df[col], min_peak_height, max_peak_height, time_series=chunk_df['fnum']) 
            for col in kinematic_cols
        }
        
        # Classify behavior for this chunk
        behavior = classify_behavior(peaks_detected)
        
        # Assign behavior to chunk
        chunk_behavior.extend([behavior] * len(chunk_df))
    
    return chunk_behavior

def apply_behavior_classification(df, min_peak_height, max_peak_height):
    """
    Apply behavior classification to each trial in the DataFrame.
    """
    df['behavior'] = np.nan  # Initialize behavior column
    
    for trial in df['filename'].unique():
        trial_df = df[df['filename'] == trial]
        
        # Process chunks to classify behavior
        chunk_behavior = process_chunks(trial_df, min_peak_height, max_peak_height)
        
        # Update the DataFrame with classified behavior
        df.loc[df['filename'] == trial, 'behavior'] = chunk_behavior
    
    return df


min_val = 50
max_val = 190

df_headless = apply_behavior_classification(headless_data, min_peak_height=None, max_peak_height=None)
df_headless['condition'] = 'e'
df_headless.loc[df_headless.date_parsed=='5.11.22', 'condition'] = 'c'

behavior_counts_headless = df_headless.groupby(['fnum', 'condition', 'stimlen','behavior']).size().reset_index(name='count')
total_counts_headless = df_headless.groupby(['fnum','condition','stimlen']).size().reset_index(name='total_count')
# Merge the behavior counts with total counts
merged_headless = pd.merge(behavior_counts_headless, total_counts_headless, on=['fnum','stimlen', 'condition'])
# Calculate the probability of each behavior at each frame
merged_headless['probability'] = merged_headless['count'] / merged_headless['total_count']
merged_headless['probability'] = merged_headless['probability'] *100


fig = plt.figure(figsize=(2.1, 1.5))
sns.barplot(data=merged_headless[merged_headless.behavior=='kicking'], 
           x='condition', 
           y='probability',
           #size=2,
          # jitter=.25
           #width=.5
           #hue='opto', 
        #palette=['#780396'],
           #errorbar=None,
           #legend=False,
           )
# plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks(np.arange(0.0, 20.1, 5))

sns.despine(trim=True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(2.1, 1.5))
sns.barplot(data=merged_headless[merged_headless.behavior=='grooming'], 
           x='condition', 
           y='probability', 
          # width=.5
           #hue='opto', 
        #palette=['#780396'],
           #errorbar=None,
           #legend=False,
           )
# plt.xticks(np.arange(0, 2.1, 0.5))
plt.yticks(np.arange(0.0, 2.1, .5))

sns.despine(trim=True)
plt.show()